In [1]:
from model import SiameseBiLSTM
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config

In [2]:
from operator import itemgetter
from keras.models import load_model
import pandas as pd

######################################<br>
########## Data Preperation ##########<br>
######################################

In [3]:
df = pd.read_csv('sample_data.csv')

In [4]:
sentences1 = list(df['sentences1'])
sentences2 = list(df['sentences2'])
is_similar = list(df['is_similar'])
del df

##################################<br>
###### Word Embedding ############<br>
##################################

creating word embedding meta data for word embedding 

In [5]:
tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'])

Embedding matrix shape: (3052, 50)
Null word embeddings: 1


In [6]:
embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

 creating sentence pairs

In [7]:
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2

########################<br>
###### Training ########<br>
########################

In [8]:
from config import siamese_config

In [9]:
class Configuration(object):
    """Dump stuff here"""

In [10]:
CONFIG = Configuration()
CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

In [11]:
siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, 
					    CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

In [12]:
best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')

Epoch 1/200
8/8 [==============================] - 50s 667ms/step - loss: 0.9292 - acc: 0.5089 - val_loss: 0.6569 - val_acc: 0.6735
Epoch 2/200
8/8 [==============================] - 1s 171ms/step - loss: 0.8218 - acc: 0.5756 - val_loss: 0.6392 - val_acc: 0.6735
Epoch 3/200
8/8 [==============================] - 2s 208ms/step - loss: 0.8078 - acc: 0.5800 - val_loss: 0.6269 - val_acc: 0.6735
Epoch 4/200
8/8 [==============================] - 1s 111ms/step - loss: 0.8605 - acc: 0.5422 - val_loss: 0.6164 - val_acc: 0.6735
Epoch 5/200
8/8 [==============================] - 1s 131ms/step - loss: 0.7678 - acc: 0.5622 - val_loss: 0.6060 - val_acc: 0.6735
Epoch 6/200
8/8 [==============================] - 1s 162ms/step - loss: 0.7255 - acc: 0.5822 - val_loss: 0.5985 - val_acc: 0.6735
Epoch 7/200
8/8 [==============================] - 1s 128ms/step - loss: 0.7728 - acc: 0.5889 - val_loss: 0.5937 - val_acc: 0.6735
Epoch 8/200
8/8 [==============================] - 1s 156ms/step - loss: 0.6949 - 

######################<br>
#### Testing #########<br>
######################

In [13]:
model = load_model(best_model_path)

In [14]:
test_sentence_pairs = [('What can make Physics easy to learn?','How can you make physics easy to learn?')]

In [15]:
test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

In [16]:
preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
results.sort(key=itemgetter(2), reverse=True)
print(results)

1/1 [==============================] - 3s 3s/step
[('What can make Physics easy to learn?', 'How can you make physics easy to learn?', 0.5556523)]


In [17]:
results[0][-1]

0.5556523